In [1]:
import random
import json
import os
from openai import OpenAI
from dotenv import load_dotenv
import gc


load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

In [2]:
from core.artwork_analysis import Artwork, ArtworkRetriever, ArtworkAnalyser, HFArtworkAnalyser

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
source = 'ars_electronica_prizewinners_ru.json'
with open(source, 'r', encoding='utf-8') as f:
    data = json.load(f)
path = 'not_posted.txt'
with open(path, 'r') as f:
    not_posted = f.readline().split(',')

key = random.choice(not_posted)

main_artwork_data = data[key]

main_artwork = Artwork(main_artwork_data)

artwork_retriever = ArtworkRetriever(source)
related_artworks = artwork_retriever.get_related_artworks(main_artwork)

In [3]:
# ChatGPT analyzer
analysis = ArtworkAnalyser(OPENAI_API_KEY)
analysis_result = analysis.analyze_artworks(main_artwork, related_artworks)

print(analysis_result)

Prompt:  Imagine you are an experienced art critic. You are presented with descriptions of three artworks. The first one is the main artwork and two related artworks are similar to the first one. Artworks also have related images, analyze their content and use this analysis in your reviews.

**Main artwork**: name: **Transfigurations**, authors: **Agi Haines**, year: **2014**, description: The human body consists of practicable elements that can be easily manipulated and engineered. Through surgical procedures our bodies can be stretched, shifted and sewn, yet still be functional. What then would stop us searching for a higher level of functionality than we have now? Especially if it may have the potential to benefit the younger, more vulnerable and more malleable generation.The exhibit consists of five animatronic sculptures of babies. Each baby shows a different design for a potential body enhancement that has been surgically implemented to benefit the child. Each modification is put

In [ ]:
import torch

del artwork_retriever
torch.cuda.empty_cache()  # Clear memory cache if using CUDA
gc.collect()

hf_analysis = HFArtworkAnalyser()
hf_analysis_result = hf_analysis.analyze_artworks(main_artwork, related_artworks)

print(hf_analysis_result)

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM
llm = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="cjpais/llava-1.6-mistral-7b-gguf", model_file="llava-v1.6-mistral-7b.Q5_K_M.gguf")


OSError: cjpais/llava-1.6-mistral-7b-gguf does not appear to have a file named config.json. Checkout 'https://huggingface.co/cjpais/llava-1.6-mistral-7b-gguf/main' for available files.

In [4]:
# from core.document_retrieval import VectorDB, Retriever

# source = 'ars_electronica_prizewinners_ru.json'
# path = 'not_posted.txt'
# 
# vector_db = VectorDB(source)
# vector_db.load()
# vector_db.create_db()
# retriever = Retriever(vector_db)
# 
# with open(source, 'r', encoding='utf-8') as f:
#     data = json.load(f)
# 
# with open(path, 'r') as f:
#     not_posted = f.readline().split(',')
# 
# key = random.choice(not_posted)
# print(f'Key is {key}')
# artwork = data[key]
# 
# print("Source:\n")
# print(artwork["description_ru"], "\n")
# 
# res = retriever.get_top_k(artwork['description_ru'], key, 2)
# 
# print("Recs:\n")
# for example in res:
#     print(example.page_content, "\n")
#     
#     
# def delete_apostrophe(text):
#     return text.replace("'", "")
# 
# art_name_main = artwork['name']
# authors_main = artwork['authors']
# year_main = artwork['year']
# description_main = delete_apostrophe(artwork['description'])
# image_main = artwork['img_list'][0]
# 
# art_name_0 = res[0].metadata['name']
# authors_0 = res[0].metadata['authors']
# year_0 = res[0].metadata['year']
# description_0 = delete_apostrophe(res[0].metadata['description'])
# image_0 = res[0].metadata['img_list'][0]
# 
# art_name_1 = res[1].metadata['name']
# authors_1 = res[1].metadata['authors']
# year_1 = res[1].metadata['year']
# description_1 = delete_apostrophe(res[1].metadata['description'])
# image_1 = res[1].metadata['img_list'][0]

In [5]:
# prompt = f'Imagine you are an experienced art critic' \
#          f'You are presented with descriptions of three artworks.' \
#          f'The first one is a main artwork and two related artworks are similar to the first one.' \
#          f'On the first step you should provide a separate review of all the artworks.' \
#          f'Artworks also have related images, analyze their content and use this analysis in your reviews.' \
#          f'On the second step summarize all of these artworks in terms of similarities and differences.' \
#          f'Main artwork: name: {art_name_main}, authors: {authors_main}, year: {year_main}, description: {description_main}' \
#          f'Related artwork 1: name: {art_name_0}, authors: {authors_0}, year: {year_0}, description: {description_0}' \
#          f'Related artwork 2: name: {art_name_1}, authors: {authors_1}, year: {year_1}, description: {description_1}' \
#          f'Give your answer in Russian, but do not translate the names of artists and artworks.' \
#          f'Do not divide description and reasoning parts, combine them into the one paragraph' \
#          f'Use Markdown to structure your answer. Make the names of artists and artworks bold.' \
#          f'Give your answer step-by-step for each artwork.'
# 
# response = client.chat.completions.create(
#     model="gpt-4-turbo",
#     messages=[
#         {
#             "role": "user",
#             "content": [
#                 {
#                     "type": "text",
#                     "text": prompt,
#                 },
#                 {
#                     "type": "image_url",
#                     "image_url":{
#                        "url": image_main,
#                        "detail": "low"
#                     },
#                 },
#                                 {
#                     "type": "image_url",
#                     "image_url":{
#                        "url": image_0,
#                        "detail": "low"
#                     },
#                 },
#                                 {
#                     "type": "image_url",
#                     "image_url":{
#                        "url": image_1,
#                        "detail": "low"
#                     },
#                 },
#             ],
#         }
#     ],
#     max_tokens=1000,
# )
# 
# print(response.choices[0])